[Node 51: SQLite mit sqlite3](http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/python2/node51.html)

Navigation:

**Next:** [Aufgaben](node52.ipynb) **Up:** [Aufgaben](node52.ipynb) **Previous:** [Aufgaben](node52.ipynb)

## SQLite with sqlite3

In contrast to other SQL database systems, SQLite is not a server but a library written in <font color=#0000ff> **C**</font>. SQL queries and statements are supported. A normal index-sequential file is used. SQLite has low CPU and memory requirements.
 

To start with, we need to define a schema, i.e. the database structure:

```bash
laptop:~/python/kurs09$ sqlite3 /tmp/blogdb.sqlite3
SQLite version 3.6.10
Enter ".help" for instructions
Enter SQL statements terminated with a ";"
sqlite> CREATE table comments (
id INTEGER PRIMARY KEY AUTOINCREMENT,
subject TEXT,
author TEXT,
text TEXT
);
sqlite> .quit
```

We can also read them in from a file:

```bash
laptop:~/python/kurs09$ rm /tmp/blogdb.sqlite3
laptop:~/python/kurs09$ sqlite3 /tmp/blogdb.sqlite3 < blogdb.schema
```

Now you can fill tables manually using <font color=#0000e6> ``INSERT``</font> statements or query tables using <font color=#0000e6> ``SELECT``</font>. Important commands in the sqlite command line interpreter:
* <font color=#0000e6> ``.help``</font>
* <font color=#0000e6> ``.schema``</font> : Displays database schema
* <font color=#0000e6> ``.dump``</font> : Shows schema and data of a database (backup)

Insert values ​​into database:
```bash
sqlite> INSERT INTO comments VALUES(
1,
'The Python Blog',
'Max Mustermann',
'How are you ?'
);
```
```bash
sqlite> INSERT INTO comments VALUES(
2,
'The Perl Blog',
'Marie Mustermann',
'How are you ?'
);
```

Query values:

```bash
sqlite> SELECT subject FROM comments ORDER by author;
The Perl Blog
The Python Blog
sqlite> SELECT subject,text FROM comments ORDER by author;
The Perl Blog|How are you ?
The Python Blog|How are you ?
```

### SQLite in Python

Connecting SQlite database in Python using <font color=#0000e6> ``sqlite3``</font> module (non-autocommit mode):

In [ ]:
import sqlite3
conn = sqlite3.connect('data/blogdb.sqlite3', isolation_level='DEFERRED')
conn

Important methods:
* <font color=#0000e6> ``close``</font> : Closes database connection. If autocommit is off, transactions without <font color=#0000e6> ``commit``</font> will not be executed.
* <font color=#0000e6> ``commit``</font> : completes open transactions and stores them in the database
* <font color=#0000e6> ``rollback``</font> : Abort the transaction.
* <font color=#0000e6> ``cursor``</font> : <font color=#0000e6> ``Cursor``</font> -Object for database query, entering, changing, deleting data.

In [ ]:
curs = conn.cursor()

Important <font color=#0000e6> ``cursor``</font> methods:
* <font color=#0000e6> ``close, execute, executemany, fetchone, fetchall``</font>

Enter data in database:

In [ ]:
curs.execute('INSERT INTO comments VALUES (? , ?, ?, ?)', (None, 'a subject','an author', 'a text'))

In [ ]:
curs.rowcount

In [ ]:
curs.execute('INSERT INTO comments VALUES (? , ?, ?, ?)', (None, 'another subject','another author', 'another text'))

In [ ]:
curs.rowcount

In [ ]:
conn.commit()

For security reasons (SQL injection vulnerability) you should always use placeholders '?' to be worked.

In [ ]:
curs.execute('SELECT * FROM comments ORDER BY id')

In [ ]:
curs.rowcount

(However, `rowcount` returns [immer -1](https://stackoverflow.com/questions/839069/cursor-rowcount-always-1-in-sqlite3-in-python3k) with `SELECT`, so it cannot be used to query the number of results found.)

In [ ]:
curs.fetchone()

In [ ]:
curs.fetchone()

In [ ]:
curs.fetchone()

In [ ]:
curs.fetchone()

In [ ]:
curs.execute('SELECT * FROM comments ORDER BY id')

In [ ]:
result=curs.fetchone()
while result is not None:
    print (result)
    result = curs.fetchone()

### Inset: SQL Vulnerability

SQL statements are vulnerable to other (potentially malicious) SQL injections, like in this [XKCD](https://xkcd.com/) comic [XKCD](https://xkcd.com/):

[![Image exploits_of_a_mom](https://imgs.xkcd.com/comics/exploits_of_a_mom.png "Her daughter is named Help I&#39;m trapped in a driver&#39;s license factory.")](https://xkcd.com/327/)

Using explicit placeholders (`?` in our `INSERT`s above) protects against this.

For memory reasons it is better to use <font color=#0000e6> ``fetchone``</font> or <font color=#0000e6> ``fetchmany``</font>.
 

Change or delete data records:

In [ ]:
curs.execute('UPDATE comments SET author=? WHERE id=?', ('me',2))

In [ ]:
curs.rowcount

In [ ]:
conn.commit()
curs.execute('SELECT * FROM comments WHERE id=?', (2,))

In [ ]:
curs.fetchone()

In [ ]:
curs.execute('DELETE FROM comments WHERE id<?', (3,))

In [ ]:
curs.rowcount

In [ ]:
conn.rollback()
curs.execute('SELECT count(*) FROM comments')

In [ ]:
curs.fetchone()

In [ ]:
curs.execute('SELECT * FROM comments ORDER BY id')

In [ ]:
curs.fetchall()

In [ ]:
conn.close()

So far only absolute basics of database usage.
<font color=#0000ff> **Relational databases**</font> offer many other features and show their strengths especially when relating different tables, but that is beyond the course here.

Good introduction: [**Software Carpentry: Databases and SQL**](http://swcarpentry.github.io/sql-novice-survey/index.html)